In [ ]:
import requests
import matplotlib.pyplot as plt
from collections import Counter

# -------------------------------
# Configuration Parameters
# -------------------------------
tenant_id    = "your_tenant_id"        # e.g. "contoso.onmicrosoft.com" or tenant GUID
client_id    = "your_client_id"
client_secret= "your_client_secret"

# Construct the tenant URL for token endpoint
tenant_url   = f"https://login.microsoftonline.com/{tenant_id}"

# -------------------------------
# Step 1: Get an Access Token
# -------------------------------
def get_access_token():
    token_url = f"{tenant_url}/oauth2/v2.0/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    # This uses Resource Owner Password Credentials flow
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
        "scope": "https://graph.microsoft.com/.default"
    }
    response = requests.post(token_url, data=data, headers=headers)
    response.raise_for_status()
    token = response.json().get('access_token')
    return token

access_token = get_access_token()
print("Access Token Retrieved Successfully")

# -------------------------------
# Step 2: Call the Microsoft Graph API to Get Users
# -------------------------------
def get_users():
    graph_url = "https://graph.microsoft.com/v1.0/users"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(graph_url, headers=headers)
    response.raise_for_status()
    return response.json()

users_data = get_users()
print("Retrieved Users Data")

# -------------------------------
# Step 3: Process the Data for Statistics
# Example: Count users by email domain.
# -------------------------------
domains = []
for user in users_data.get('value', []):
    email = user.get('mail') or user.get('userPrincipalName')
    if email:
        domain = email.split('@')[-1]
        domains.append(domain)

domain_counts = Counter(domains)
print("User counts per domain:", domain_counts)

# -------------------------------
# Step 4: Plot the Statistics
# -------------------------------
plt.figure(figsize=(8, 4))
plt.bar(domain_counts.keys(), domain_counts.values())
plt.xlabel("Email Domain")
plt.ylabel("Number of Users")
plt.title("User Distribution by Email Domain")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
